In [6]:
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Load dataset from CSV
dataset_file = "clean2.data.csv"  # Update with the path to your combined CSV file
file_dataset = open(dataset_file, mode='r')
csvreader_dataset = csv.reader(file_dataset)
next(csvreader_dataset)  # Skip headers

# Initialize lists to store training and test data
train_data = []
test_data = []

# Read data from CSV and separate into training and test data
for i, row in enumerate(csvreader_dataset):
    if i < 3600:
        train_data.append(row)
    else:
        test_data.append(row)

# Close CSV file
file_dataset.close()

# Convert lists to numpy arrays
train_data = np.array(train_data)
test_data = np.array(test_data)

# Separate features and labels for training data
X_train = train_data[:, 2:-1].astype(float)  # Assuming features start from column index 2 and end before the last column
y_train = train_data[:, -1]
rIndex = np.random.permutation(len(train_data))
X_train = X_train[rIndex]
y_train = y_train[rIndex]
y_train = np.where(y_train=="1.", 1, -1)

# Separate features and labels for test data
X_test = test_data[:, 2:-1].astype(float)
y_test = test_data[:, -1]
y_test = np.where(y_test=="1.", 1, -1) 


# Display shapes of datasets
print("Shape of training data (X_train):", X_train.shape)
print("Shape of training labels (y_train):", y_train.shape)
print("Shape of testing data (X_test):", X_test.shape)
print("Shape of testing labels (y_test):", y_test.shape)


ValueError: invalid literal for int() with base 10: '0.'

In [2]:
print(y_train)
print(y_test)

[-1 -1 -1 ... -1 -1 -1]
[-1 -1 -1 ... -1 -1 -1]


In [3]:
# Define PercentCorrect function
def PercentCorrect(inputs, targets, weights):
    N = len(targets)
    targets = targets.astype(np.float64)
    nCorrect = 0
    for n in range(N):
        oneInput = inputs[n, :]
        if(targets[n] * np.dot(oneInput, weights) > 0):
            nCorrect += 1
    return 100 * nCorrect / N

In [4]:
# Randomly initialize weights
w = np.random.randn(X_train.shape[1])
print("Randomly generated weight vector (w):", w)

# Print initial percentage correct
print("Initial Percentage Correct: %6.2f" % (PercentCorrect(X_train, y_train, w)))

# Set parameters for training
MaxIter = 2000
alpha = 0.0001
P_train = np.zeros(MaxIter)
P_test = np.zeros(MaxIter)

# Training loop
Ntrain = X_train.shape[0]
Ntest = X_test.shape[0]

for iter in range(MaxIter):
    r = np.floor(np.random.rand() * Ntrain).astype(int)
    x = X_train[r, :]

    if y_train[r] * np.dot(x, w) < 0:
        w += alpha * y_train[r] * x

    P_train[iter] = PercentCorrect(X_train, y_train, w)
    P_test[iter] = PercentCorrect(X_test, y_test, w)

# Print percentage correct after training
print("Percentage Correct After Training: %6.2f %6.2f" % (PercentCorrect(X_train, y_train, w), PercentCorrect(X_test, y_test, w)))


Randomly generated weight vector (w): [ 0.85682687  1.80126352 -1.40314106 -1.49703777  0.04420332 -0.03484543
  1.01812766  0.16996334 -1.8513496  -1.33607211 -0.43429325 -0.28697551
  1.46962012 -0.55321996 -0.37628216 -0.51950743 -2.27699398  0.59508259
  1.20716268  0.04191969 -0.65429985 -0.485753   -0.52950468 -0.69531487
  2.1030207   0.95068824 -0.68137611  0.12089744 -0.47903021  0.95693207
 -1.47893515  1.22227053 -0.2617005  -0.53180298 -0.1811652   0.3609482
  0.52316125 -0.84940391 -0.37280067 -0.04794134  0.41669512 -0.84608357
 -0.38254869 -1.68004501 -1.09963062  0.94245357 -0.55490458  0.16031281
  0.0420231   1.04275915 -1.00254742  0.12228954 -0.84329008 -0.14757709
  1.36872586  1.49861698 -0.93666581  0.65940122  0.61422422 -0.33047898
 -1.0203218   2.42504234 -1.36370657 -0.55070016  0.54471147 -0.93641188
  0.3205577   0.05617774 -0.57656985 -0.33946526  1.37698075  0.05361287
  0.80775541 -0.68670083 -0.46553188  0.37674656  1.05902874  0.07747134
  0.85145642  